In [5]:
from PIL import Image
from math import sqrt

In [21]:

img = Image.open("enhance_0.jpg")


In [22]:
def convolutionMask(img,pixels,position,mask):
    valueList = []
    pX = position[0]
    pY = position[1]
    for i in range(3):
        mY = i-1+pY
        #if mY < 0 or mY >= img.height:
        #    continue
        
        for j in range(3):
            mX = j-1 + pX
            if mX < 0 or mX >= img.width or mY < 0 or mY >= img.height:
                valueList.append([pixels[pX,pY][p] * mask[i][j] for p in range(3)] )
                continue    
                
            value =  [pixels[mX,mY][p] * mask[i][j] for p in range(3)] 
            valueList.append(value)
    #print(valueList)
    
    r = sum([pixel[0] for pixel in valueList])
    g = sum([pixel[1] for pixel in valueList])
    b = sum([pixel[2] for pixel in valueList])
    
    result= (r,g,b)
    #if(result<0):
    #    return 0 
    #elif(result>255):
    #    return 255 
    #else: 
    #    return result
    return result

In [23]:
#img.save("source_greyscale.jpg")

In [24]:
pixels = img.load()
mask = [[-1,-1,-1],
        [-1,8,-1],
        [-1,-1,-1]]
print(convolutionMask(img,pixels,(0,0),mask))
print(pixels[99,99],pixels[99,100],pixels[99,101],pixels[100,99],pixels[100,100],pixels[100,101],pixels[101,99],pixels[101,100],pixels[101,101])
print(convolutionMask(img,pixels,[100,100],mask))

(2, 2, 2)
(227, 202, 37) (227, 202, 39) (227, 202, 40) (227, 202, 37) (226, 201, 38) (226, 201, 38) (227, 202, 37) (226, 201, 38) (225, 200, 37)
(-4, -4, 1)


In [25]:
def createImageEdge(img):
    output =Image.new('RGB',(img.width,img.height),'white')
    outputPixels = output.load()
    #print(img.width,img.height)
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            value = convolutionMask(img,pixels,(i,j),mask)
            #print(value)
            outputPixels[i,j] = value
    return output
    


In [26]:
def mergeImage(img1,img2):
    pixels1 = img1.load()
    pixels2 = img2.load()
    
    output = Image.new('RGB',(img1.width,img1.height),"white")
    outputPixel = output.load()
    for j in range(img1.height):
        for i in range(img1.width):
            value = [pixels1[i,j][p] + pixels2[i,j][p] for p in range(3)]
            outputPixel[i,j] = tuple(value)
            
    return output
            

In [12]:
edgeImg = createImageEdge(img)
edgeImg.save("RGBprocess1.jpg")

In [13]:
mergeImg = mergeImage(img,edgeImg)
mergeImg.save('RGBprocess2.jpg')

In [14]:
def createSobelFilterImage(img):
    maskX = [[-1,0,1],
             [-2,0,2],
             [-1,0,1]]
    maskY = [[-1,-2,-1],
             [0,0,0],
             [1,2,1]]
    
    output = Image.new('RGB',(img.width,img.height),"white")
    outputPixel = output.load()
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            gX = convolutionMask(img,pixels,(i,j),maskX)
            gY = convolutionMask(img,pixels,(i,j),maskY)
            
            value = [int(sqrt(gX[i]*gX[i] + gY[i]*gY[i])) for i in range(3)]
            outputPixel[i,j] = tuple(value)
    return output

In [15]:
sobelImg = createSobelFilterImage(img)
sobelImg.save('RGBprocess3.jpg')

In [18]:
def advangeImage(img):
    mask = [[0.125,0.125,0.125],
        [0.125,0.125,0.125],
        [0.125,0.125,0.125]]
    output = Image.new('RGB',(img.width,img.height),"white")
    outputPixel = output.load()
    
    pixels = img.load()
    for j in range(img.height):
        for i in range(img.width):
            value = convolutionMask(img,pixels,(i,j),mask)
            outputPixel[i,j] = (int(value[0]),int(value[1]),int(value[2]))
    return output

In [19]:
advancedImg = advangeImage(sobelImg)
advancedImg.save('RGBprocess4.jpg')

In [27]:
def normalizeAndMerge(normalizeImg,inputImg):
    pixelsNormal = normalizeImg.load()
    inputPixels = inputImg.load()
    
    output = Image.new('RGB',(normalizeImg.width,normalizeImg.height),"white")
    outputPixel = output.load()
    
    for j in range(img.height):
        for i in range(img.width):
            
            value = [int(pixelsNormal[i,j][p] / 255 * inputPixels[i,j][p]) for p in range(3)]
            outputPixel[i,j] = tuple(value)
    return output
    

In [28]:
noramlizedImg = normalizeAndMerge(advancedImg,edgeImg)
noramlizedImg.save('RGBprocess5.jpg')

In [29]:
finalImg = mergeImage(noramlizedImg,img)
finalImg.save('RGBprocess6.jpg')